In [142]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
import csv
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [143]:
dataDir = ".\spells"
data = []
#Read all data from csv file in same directory (same spell)

for folder in os.listdir(dataDir):
    for filename in os.listdir(dataDir + '\\' + folder):
        f = os.path.join(dataDir + '\\' + folder, filename)
        if os.path.isfile(f):
            #File
            with open(f, 'r') as f:
                # Use the csv reader to read the file
                reader = csv.reader(f)
                # Store the data in a list
                data.append(list(reader))
            
# Print the data
data


[[['1.58', '2.06', '7.71', '0.61', '0.31', '0.26'],
  ['0.66', '2.05', '7.91', '0.37', '0.79', '1.96'],
  ['-3.71', '-0.35', '13.27', '0.41', '1.41', '1.3'],
  ['-5.63', '-0.81', '17.72', '-2.37', '1.2', '-1.42'],
  ['-2.0', '1.13', '11.6', '-5.37', '-1.16', '-2.51'],
  ['-0.07', '-1.0', '4.97', '-3.61', '1.29', '-0.54'],
  ['0.02', '-3.53', '1.07', '-0.43', '0.33', '-0.21'],
  ['-2.14', '-2.28', '-1.52', '1.8', '0.33', '0.38'],
  ['-3.45', '-1.66', '3.03', '6.79', '0.32', '-1.21'],
  ['-2.62', '4.38', '24.15', '4.42', '0.47', '-0.52'],
  ['-1.33', '2.76', '14.72', '0.28', '-0.13', '-0.02'],
  ['-1.84', '2.97', '8.99', '-0.2', '-0.34', '0.28'],
  ['-1.93', '3.01', '9.02', '-0.36', '-0.44', '0.18'],
  ['-1.5', '2.84', '8.81', '0.05', '-0.16', '0.18'],
  ['-0.97', '3.14', '9.26', '-0.18', '0.18', '-0.04']],
 [['0.59', '1.43', '7.96', '-0.43', '0.35', '0.17'],
  ['-1.04', '0.89', '8.66', '0.38', '0.04', '1.18'],
  ['-1.24', '0.78', '9.95', '0.26', '0.15', '0.36'],
  ['-3.56', '0.39', '10.

In [144]:
#Find longest vector
longest_len = max([len(vector) for vector in data])
print("Longest length: ", longest_len)

Longest length:  51


In [170]:
#Apply padding to vectors sequences less than max length 
for vectors in data:
    if len(vectors) < longest_len:
        difference = longest_len - len(vectors)
        array = np.zeros((difference, 6))
        list_of_arrays = array.tolist()
        vectors.extend(list_of_arrays)
        

In [266]:
#Additional parameters
batch_size = 10           #length of files
sequence_length = longest_len    #Amount of data vectors in a file
num_features = 6           #Length of data vectors
n_hidden = 128
num_classes = 2

data = np.array(data, dtype=float)
#print(data)


labels = [np.zeros(51) for i in range(5)]
labels.extend([np.ones(51) for i in range(5)])


In [268]:
# Preprocess the data
# Split the data into training and test sets
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2)


# Normalize the data
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)


# Choose the RNN architecture
model = tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(32, input_shape=(sequence_length, num_features), return_sequences=True))  # LSTM layer with 32 units and return_sequences=True
model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(2, activation='softmax')))  # TimeDistributed wrapper around Dense output layer with 2 units and softmax activation

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
x_train = x_train.reshape(-1,sequence_length,num_features)
model.fit(x_train, y_train, epochs=10, batch_size=batch_size)

# Evaluate the model
loss, accuracy = model.evaluate(x_test, y_test)
print(f'Test loss: {loss:.4f}')
print(f'Test accuracy: {accuracy:.4f}')

# Use the model to classify new sequences of data
predictions = model.predict(new_data)


ValueError: Found array with dim 3. StandardScaler expected <= 2.

In [287]:
# Define the model architecture
model = tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(32, input_shape=(51, 6), return_sequences=True))  # LSTM layer with 32 units and return_sequences=True
model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(2, activation='softmax')))  # TimeDistributed wrapper around Dense output layer with 2 units and softmax activation

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Generate dummy data
#ESTOS SON LOS TAMAÑOS QUE FUNCIONAN

labels = [np.zeros(51) for i in range(5)]
labels.extend([np.ones(51) for i in range(5)])
# One-hot encode the labels
labels = tf.keras.utils.to_categorical(labels, num_classes=2)

# Train the model
model.fit(data, labels, epochs=10, batch_size=4)

# Evaluate the model
loss, accuracy = model.evaluate(data, labels)
print(f'Test loss: {loss:.4f}')
print(f'Test accuracy: {accuracy:.4f}')



Epoch 1/10
3/3 [==============================] - 2s 13ms/step - loss: 0.6940 - accuracy: 0.4784
Epoch 2/10
3/3 [==============================] - 0s 12ms/step - loss: 0.6972 - accuracy: 0.4588
Epoch 3/10
3/3 [==============================] - 0s 13ms/step - loss: 0.6921 - accuracy: 0.5216
Epoch 4/10
3/3 [==============================] - 0s 12ms/step - loss: 0.6935 - accuracy: 0.5020
Epoch 5/10
3/3 [==============================] - 0s 13ms/step - loss: 0.6907 - accuracy: 0.5863
Epoch 6/10
3/3 [==============================] - 0s 13ms/step - loss: 0.6904 - accuracy: 0.5824
Epoch 7/10
3/3 [==============================] - 0s 13ms/step - loss: 0.6898 - accuracy: 0.6020
Epoch 8/10
3/3 [==============================] - 0s 13ms/step - loss: 0.6910 - accuracy: 0.5333
Epoch 9/10
3/3 [==============================] - 0s 12ms/step - loss: 0.6879 - accuracy: 0.6157
Epoch 10/10
1/1 [==============================] - 0s 401ms/step - loss: 0.6860 - accuracy: 0.5922
Test loss: 0.6860
Test accur

In [291]:
new_data = []

with open(dataDir + '\\testing\\test1.csv', 'r') as f:
    # Use the csv reader to read the file
    reader = csv.reader(f)
    # Store the data in a list
    new_data.append(list(reader))



if len(new_data) < longest_len:
        difference = longest_len - len(new_data)
        array = np.zeros((difference, 6))
        list_of_arrays = array.tolist()
        new_data.extend(list_of_arrays)
        
  
#Prediccion
#redictions = model.predict(new_data)

1
